In [1]:
import numpy as np
import pandas as pd

In [2]:
# Upload the dataset
listing_path = 'listings.csv'
listing_data = pd.read_csv(listing_path)
print(listing_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8021 entries, 0 to 8020
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            8021 non-null   int64  
 1   listing_url                                   8021 non-null   object 
 2   scrape_id                                     8021 non-null   int64  
 3   last_scraped                                  8021 non-null   object 
 4   source                                        8021 non-null   object 
 5   name                                          8021 non-null   object 
 6   description                                   7663 non-null   object 
 7   neighborhood_overview                         2757 non-null   object 
 8   picture_url                                   8019 non-null   object 
 9   host_id                                       8021 non-null   i

In [3]:
# Drop rows where property_type has less than 35 value counts
property_counts = listing_data['property_type'].value_counts()
print(listing_data.shape)
to_keep = property_counts[property_counts >= 35].index
listing_data = listing_data[listing_data['property_type'].isin(to_keep)]
print(listing_data.shape)

(8021, 75)
(7749, 75)


In [4]:
columns_to_drop = [
    'listing_url', 'source' ,'scrape_id', 'host_url', 'picture_url', 'host_thumbnail_url', 
    'host_picture_url', 'last_scraped', 'calendar_last_scraped', 'calendar_updated', 
    'neighbourhood_group_cleansed', 'host_about', 'host_since', 'host_location', 
    'host_name', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 
    'host_neighbourhood', 'host_has_profile_pic', 'license', 'minimum_minimum_nights', 
    'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 
    'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'host_total_listings_count', 'host_verifications', 'neighbourhood',
    'last_review', 'first_review', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms', 'neighborhood_overview', 'room_type', 'name','host_id', 'id'
]

# Drop features that are not useful for the analysis
listing_data = listing_data.drop(columns=columns_to_drop)
print(listing_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 7749 entries, 0 to 8020
Data columns (total 35 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   description                     7407 non-null   object 
 1   host_is_superhost               7680 non-null   object 
 2   host_listings_count             7749 non-null   int64  
 3   host_identity_verified          7749 non-null   object 
 4   neighbourhood_cleansed          7749 non-null   object 
 5   latitude                        7749 non-null   float64
 6   longitude                       7749 non-null   float64
 7   property_type                   7749 non-null   object 
 8   accommodates                    7749 non-null   int64  
 9   bathrooms                       5625 non-null   float64
 10  bathrooms_text                  7741 non-null   object 
 11  bedrooms                        7052 non-null   float64
 12  beds                            5625 no

In [5]:
# Show percentage of null values in each column
null_percentage = listing_data.isnull().mean() * 100
print(null_percentage)
print(listing_data.shape)

description                        4.413473
host_is_superhost                  0.890437
host_listings_count                0.000000
host_identity_verified             0.000000
neighbourhood_cleansed             0.000000
latitude                           0.000000
longitude                          0.000000
property_type                      0.000000
accommodates                       0.000000
bathrooms                         27.409988
bathrooms_text                     0.103239
bedrooms                           8.994709
beds                              27.409988
amenities                          0.000000
price                             27.332559
minimum_nights                     0.000000
maximum_nights                     0.000000
has_availability                   3.923087
availability_30                    0.000000
availability_60                    0.000000
availability_90                    0.000000
availability_365                   0.000000
number_of_reviews               

In [6]:
listing_data['reviews_per_month'] = listing_data['reviews_per_month'].fillna(0)
review_score_columns = ['review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value']
boolean_columns = ['host_is_superhost', 'host_identity_verified', 'has_availability', 'instant_bookable']
# Group by neighbourhood_cleansed and room_type to fill NaN values
listing_data['price'] = listing_data['price'].replace('[€$,]', '', regex=True).astype(float)
listing_data['beds'] = listing_data.groupby(['neighbourhood_cleansed', 'property_type'])['beds'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 1))
# Drop rows where all prices in the group are NaN
listing_data = listing_data.groupby(['neighbourhood_cleansed', 'property_type']).filter(lambda x: x['price'].notna().any())                                                            
listing_data['price'] = listing_data.groupby(['neighbourhood_cleansed', 'property_type'])['price'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x.mean()))     

# Fill missing review scores with mean
for column in review_score_columns:
    listing_data = listing_data.groupby(['neighbourhood_cleansed', 'property_type']).filter(lambda x: x[review_score_columns].notna().any().all())
    listing_data[column] = listing_data.groupby(['neighbourhood_cleansed', 'property_type'])[column].transform(lambda x: x.fillna(x.mean()))

# Transform boolean columns
for column in boolean_columns:
    listing_data[column] = listing_data[column].map({'t': 1, 'f': 0})

# Split bathrooms_text into num_bathrooms and type_bathroom
listing_data[['num_bathrooms', 'type_bathroom']] = listing_data['bathrooms_text'].str.extract(r'(\d*\.?\d+)\s*(.*)')
listing_data = listing_data.dropna(subset=['type_bathroom'])
listing_data['num_bathrooms'] = listing_data['num_bathrooms'].astype(float)

# Fill missing bathrooms with num_bathrooms and vice versa
listing_data['bathrooms'] = listing_data.apply(lambda row: row['num_bathrooms'] if pd.isna(row['bathrooms']) and not pd.isna(row['num_bathrooms']) else row['bathrooms'], axis=1)
listing_data['num_bathrooms'] = listing_data.apply(lambda row: row['bathrooms'] if pd.isna(row['num_bathrooms']) and not pd.isna(row['bathrooms']) else row['num_bathrooms'], axis=1)

# Drop rows where both bathrooms and num_bathrooms are NaN
listing_data = listing_data.dropna(subset=['bathrooms', 'num_bathrooms'], how='all')

# Transform type_bathroom values
listing_data['type_bathroom'] = listing_data['type_bathroom'].replace({'bath': 'private', 'private bath': 'private', 'shared bath': 'shared'})

# Fill missing bedrooms with default value 1
listing_data['bedrooms'] = listing_data['bedrooms'].fillna(1)

In [7]:
# Fill NaN values in 'host_is_superhost' with 0 (False)
listing_data['host_is_superhost'] = listing_data['host_is_superhost'].fillna(0)

# Fill NaN values in 'description' with an empty string
listing_data['description'] = listing_data['description'].fillna("")

# Set 'has_availability' to 1 (True) if 'availability_30' is greater than 25, otherwise 0 (False)
listing_data['has_availability'] = listing_data['availability_30'].apply(lambda x: 1 if x >= 25 else 0)

In [8]:
print(listing_data['price'].isnull().sum())

0


In [9]:
null_percentage = listing_data.isnull().mean() * 100
print(null_percentage)
print(listing_data.shape)

description                       0.0
host_is_superhost                 0.0
host_listings_count               0.0
host_identity_verified            0.0
neighbourhood_cleansed            0.0
latitude                          0.0
longitude                         0.0
property_type                     0.0
accommodates                      0.0
bathrooms                         0.0
bathrooms_text                    0.0
bedrooms                          0.0
beds                              0.0
amenities                         0.0
price                             0.0
minimum_nights                    0.0
maximum_nights                    0.0
has_availability                  0.0
availability_30                   0.0
availability_60                   0.0
availability_90                   0.0
availability_365                  0.0
number_of_reviews                 0.0
number_of_reviews_ltm             0.0
number_of_reviews_l30d            0.0
review_scores_rating              0.0
review_score

In [10]:
# Compute the 99th percentile of the price for each neighbourhood and property type
price_99_percentile = listing_data.groupby(['neighbourhood_cleansed', 'property_type'])['price'].quantile(0.99)

# Filter out the rows where the price is higher than 99th percentile
listing_data = listing_data[listing_data.apply(
    lambda row: row['price'] <= price_99_percentile.loc[row['neighbourhood_cleansed'], row['property_type']],
    axis=1
)]
print(f"{listing_data.shape}")

(7391, 37)


In [11]:
import ast
# List of features to check for
features = ['wifi', 'pool', 'gym', 'bluetooth', 'refrigerator', 'stove', 'oven', 'TV', 'grill', 'parking', 'garage', 'kitchen', 'fireplace', 'housekeeping', 'console', 'exercise', 'coffee maker', 'clothing storage']
listing_data['amenities'] = listing_data['amenities'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
# Function to check if a feature is present in the amenities
def has_feature(amenities, feature):
    return int(any(feature.lower() in amenity.lower() for amenity in amenities))

# Create new features
for feature in features:
    listing_data[f'has_{feature.replace(" ", "_")}'] = listing_data['amenities'].apply(lambda x: has_feature(x, feature))

# Count the number of rows for each new feature
feature_counts = {f'has_{feature.replace(" ", "_")}': listing_data[f'has_{feature.replace(" ", "_")}'].sum() for feature in features}
# Print the feature counts sorted in decreasing order
sorted_feature_counts = dict(sorted(feature_counts.items(), key=lambda item: item[1], reverse=True))
print(sorted_feature_counts)

{'has_wifi': np.int64(7002), 'has_kitchen': np.int64(6742), 'has_TV': np.int64(4823), 'has_parking': np.int64(4428), 'has_refrigerator': np.int64(4421), 'has_stove': np.int64(3596), 'has_coffee_maker': np.int64(3357), 'has_oven': np.int64(3151), 'has_clothing_storage': np.int64(2728), 'has_grill': np.int64(659), 'has_exercise': np.int64(584), 'has_bluetooth': np.int64(329), 'has_garage': np.int64(272), 'has_fireplace': np.int64(214), 'has_console': np.int64(177), 'has_pool': np.int64(159), 'has_gym': np.int64(158), 'has_housekeeping': np.int64(78)}


In [12]:
# Drop the 'amenities' column
listing_data = listing_data.drop(columns=['amenities'])
print(listing_data.columns)

Index(['description', 'host_is_superhost', 'host_listings_count',
       'host_identity_verified', 'neighbourhood_cleansed', 'latitude',
       'longitude', 'property_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'price', 'minimum_nights',
       'maximum_nights', 'has_availability', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'instant_bookable',
       'calculated_host_listings_count', 'reviews_per_month', 'num_bathrooms',
       'type_bathroom', 'has_wifi', 'has_pool', 'has_gym', 'has_bluetooth',
       'has_refrigerator', 'has_stove', 'has_oven', 'has_TV', 'has_grill',
       'has_parking', 'has_garage', 'has_kitchen', 'has

In [13]:
columns_to_check = [
    'number_of_reviews',
    'number_of_reviews_ltm',
    'number_of_reviews_l30d',
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value'
]

results = {}

# Check for negative values and values equal to 0 and store the results in the dictionary in order to print them

for column in columns_to_check:
    if column in listing_data.columns: 
        anomalies = {
            'negative_values': (listing_data[column] > 0).sum(),  
            'greater_than_5': (listing_data[column] == 0).sum(), 
        }
        results[column] = anomalies
    else:
        results[column] = 'Column not found in the dataset'

for column, anomalies in results.items():
    print(f"{column}:")
    if isinstance(anomalies, dict):
        print(f"  Greater than 0: {anomalies['negative_values']}")
        print(f"  Equal to 0 : {anomalies['greater_than_5']}")
    else:
        print(f"  {anomalies}")
    print()


number_of_reviews:
  Greater than 0: 5496
  Equal to 0 : 1895

number_of_reviews_ltm:
  Greater than 0: 4152
  Equal to 0 : 3239

number_of_reviews_l30d:
  Greater than 0: 2000
  Equal to 0 : 5391

review_scores_rating:
  Greater than 0: 7391
  Equal to 0 : 0

review_scores_accuracy:
  Greater than 0: 7391
  Equal to 0 : 0

review_scores_cleanliness:
  Greater than 0: 7391
  Equal to 0 : 0

review_scores_checkin:
  Greater than 0: 7390
  Equal to 0 : 1

review_scores_communication:
  Greater than 0: 7391
  Equal to 0 : 0

review_scores_location:
  Greater than 0: 7391
  Equal to 0 : 0

review_scores_value:
  Greater than 0: 7391
  Equal to 0 : 0



In [14]:
# Transform the columns to boolean values
listing_data['number_of_reviews_ltm'] = listing_data['number_of_reviews_ltm'] > 0
listing_data['number_of_reviews_l30d'] = listing_data['number_of_reviews_l30d'] > 0

In [15]:
score_columns = [
    'review_scores_rating',
    'review_scores_accuracy',
    'review_scores_cleanliness',
    'review_scores_checkin',
    'review_scores_communication',
    'review_scores_location',
    'review_scores_value'
]

# Compute the mean of the review scores
listing_data['review_scores_mean'] = listing_data[score_columns].mean(axis=1)
# Drop the individual review scores
listing_data.drop(columns=score_columns, inplace=True)

In [16]:
# Drop rows with NaN values in 'review_scores_mean'
listing_data = listing_data.drop(columns=['bathrooms', 'bathrooms_text'])
print(listing_data[['num_bathrooms', 'type_bathroom']])

      num_bathrooms type_bathroom
0               1.0       private
1               1.0       private
2               1.0       private
3               1.0       private
4               1.0        shared
...             ...           ...
8015            1.0       private
8016            1.0       private
8018            1.0       private
8019            1.0        shared
8020            1.0        shared

[7391 rows x 2 columns]


In [17]:
# One-hot encoding for neighbourhood_cleansed and property_type
listing_data = pd.get_dummies(listing_data, columns=['neighbourhood_cleansed', 'property_type',  'type_bathroom'], drop_first=True)
print(listing_data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 7391 entries, 0 to 8020
Data columns (total 81 columns):
 #   Column                                                                         Non-Null Count  Dtype  
---  ------                                                                         --------------  -----  
 0   description                                                                    7391 non-null   object 
 1   host_is_superhost                                                              7391 non-null   float64
 2   host_listings_count                                                            7391 non-null   int64  
 3   host_identity_verified                                                         7391 non-null   int64  
 4   latitude                                                                       7391 non-null   float64
 5   longitude                                                                      7391 non-null   float64
 6   accommodates                 

In [18]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Load your dataset
# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to analyze sentiment
def get_sentiment(text):
    sentiment_score = analyzer.polarity_scores(text)
    return sentiment_score['compound']  # Returns the sentiment score

# Apply the sentiment analysis function to the description column
# listing_data['sentiment_score'] = listing_data['description'].apply(get_sentiment)
listing_data = listing_data.drop(columns=['description'])

# We tried to add sentiment_score feature, but we did not obtain any significant improvement in the models performance, so we discarded it.

In [19]:
from sklearn.preprocessing import StandardScaler

# List of columns to normalize
columns_to_normalize = [
    'host_listings_count', 'latitude', 'longitude', 'accommodates', 'bedrooms', 'beds', 
    'minimum_nights', 'maximum_nights', 'availability_30', 'availability_60', 
    'availability_90', 'availability_365', 'number_of_reviews', 
    'calculated_host_listings_count', 'reviews_per_month', 'num_bathrooms', 
    'review_scores_mean'
]

# Initialize the StandardScaler
scaler = StandardScaler()

# Apply z-score normalization
listing_data[columns_to_normalize] = scaler.fit_transform(listing_data[columns_to_normalize])

# Print mean and standard deviation for each feature
for column in columns_to_normalize:
    mean = listing_data[column].mean()
    std = listing_data[column].std()
    print(f"{column}: mean = {mean:.2f}, std = {std:.2f}")

print(listing_data[['latitude', 'longitude']])

host_listings_count: mean = -0.00, std = 1.00
latitude: mean = 0.00, std = 1.00
longitude: mean = 0.00, std = 1.00
accommodates: mean = -0.00, std = 1.00
bedrooms: mean = -0.00, std = 1.00
beds: mean = -0.00, std = 1.00
minimum_nights: mean = 0.00, std = 1.00
maximum_nights: mean = 0.00, std = 1.00
availability_30: mean = -0.00, std = 1.00
availability_60: mean = -0.00, std = 1.00
availability_90: mean = 0.00, std = 1.00
availability_365: mean = 0.00, std = 1.00
number_of_reviews: mean = -0.00, std = 1.00
calculated_host_listings_count: mean = -0.00, std = 1.00
reviews_per_month: mean = -0.00, std = 1.00
num_bathrooms: mean = 0.00, std = 1.00
review_scores_mean: mean = -0.00, std = 1.00
      latitude  longitude
0    -0.967912  -1.513552
1    -0.741470   1.550787
2     0.481865   0.044699
3    -0.043762  -2.003714
4    -1.233073  -0.731322
...        ...        ...
8015  2.745108  -0.104148
8016  0.531808   0.013467
8018  1.765100  -0.242996
8019 -0.015604  -0.412758
8020  0.413424  -0

In [20]:
# check for NaN values
columns_with_nan = listing_data.columns[listing_data.isna().any()]

for column in columns_with_nan:
    num_nan = listing_data[column].isna().sum()
    print(f"'{column}': {num_nan} NaN values")

In [21]:
listing_data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 7391 entries, 0 to 8020
Data columns (total 80 columns):
 #   Column                                                                         Non-Null Count  Dtype  
---  ------                                                                         --------------  -----  
 0   host_is_superhost                                                              7391 non-null   float64
 1   host_listings_count                                                            7391 non-null   float64
 2   host_identity_verified                                                         7391 non-null   int64  
 3   latitude                                                                       7391 non-null   float64
 4   longitude                                                                      7391 non-null   float64
 5   accommodates                                                                   7391 non-null   float64
 6   bedrooms                     

In [22]:
listing_data.to_csv('listings_clean.csv', index=False)